In [2]:
import pandas as pd
import numpy as np

Reading in Data

In [3]:
movies = pd.read_csv("../data/ml-1m/movies.dat", sep="::", header = None)
ratings = pd.read_csv("../data/ml-1m/ratings.dat", sep="::", header = None)
users = pd.read_csv("../data/ml-1m/users.dat", sep="::", header = None)

c:\python\python36\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
c:\python\python36\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
c:\python\python36\lib\site-packages\ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel packag

In [4]:
print('shape of movies-matrix:', movies.shape)
print('shape of ratings-matrix:', ratings.shape)
print('shape of users-matrix:', users.shape)

shape of movies-matrix: (3883, 3)
shape of ratings-matrix: (1000209, 4)
shape of users-matrix: (6040, 5)


In [5]:
movies.columns = ["MovieID","MovieTitle","Genre"]
movies.head(5)


,MovieID,MovieTitle,Genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
ratings.columns = ["UserID","MovieID","Rating","TimeStamp"]
ratings.head(5)

,UserID,MovieID,Rating,TimeStamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [7]:
users.columns = ["UserID","Sex","AgeGroup","OccupationGroup","ZipCode"]
users.head(5)

,UserID,Sex,AgeGroup,OccupationGroup,ZipCode
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [8]:
global_average_rating = ratings.mean(0)[2]
global_average_rating

3.581564453029317

In [9]:
global_avg_user = ratings.groupby(["UserID"])["Rating"].mean()

global_avg_user.head(5)

UserID
1    4.188679
2    3.713178
3    3.901961
4    4.190476
5    3.146465
Name: Rating, dtype: float64

In [11]:
global_avg_movie = ratings.groupby(["MovieID"])["Rating"].mean()
global_avg_movie.head(5)

MovieID
1    4.146846
2    3.201141
3    3.016736
4    2.729412
5    3.006757
Name: Rating, dtype: float64

In [123]:


def DesignMat(Ratings_df):
    
    global_avg_user = ratings.groupby(["UserID"])["Rating"].mean()
    global_avg_movie = ratings.groupby(["MovieID"])["Rating"].mean()
    
    Y = Ratings_df["Rating"]
    X1 = global_avg_user[Ratings_df["UserID"]]
    X2 = global_avg_movie[Ratings_df["MovieID"]]
    X3 = np.repeat(1,len(Y))
    X = np.vstack([X1,X2,X3]).T
    return([X,Y])
    
[X,Y] = DesignMat(ratings)  

Betas = np.linalg.lstsq(X,Y)
Betas[0]

#np.dot(X,Betas[0])

c:\python\python36\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


array([ 0.78212853,  0.8757397 , -2.35619748])

In [122]:
np.random.seed(610)
Nfolds = 5

#Create folds grouping vector
Nrep = ratings.shape[0] // Nfolds
a = np.repeat(np.arange(Nfolds),Nrep)
b = np.arange(len(Y) % Nfolds)
folds_vec = np.concatenate([a,b])
np.random.shuffle(folds_vec)

#initialize error vecs
err_train = np.zeros(Nfolds)
err_test = np.zeros(Nfolds)

for fold in range(Nfolds):
    ind = folds_vec == fold
    ratings_test = ratings.loc[ind]
    ratings_train = ratings.loc[~ind]
    
    [X,Y] = DesignMat(ratings_train)
    betas = np.linalg.lstsq(X,Y)[0]
    Ypred = np.dot(X, betas)
    #improving Ypred
    Ypred[Ypred>5] = 5
    Ypred[Ypred<1] = 1
    
    
    err_train[fold] = np.sqrt(np.mean( (Y-Ypred)**2))
    
    [X,Y] = DesignMat(ratings_test)
    Ypred = np.dot(X, betas)
    #improving Ypred
    Ypred[Ypred>5] = 5
    Ypred[Ypred<1] = 1
    
    err_test[fold] = np.sqrt(np.mean( (Y-Ypred)**2))
    
    print("Fold " + str(fold) + ": RMSE_train=" + str(err_train[fold]) + "; RMSE_test=" + str(err_test[fold]))
    
    
    
print("\n")
print("Mean error on TRAIN: " + str(np.mean(err_train)))
print("Mean error on  TEST: " + str(np.mean(err_test)))

#TO DO: Fallback implementation??


c:\python\python36\lib\site-packages\ipykernel_launcher.py:21: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


Fold 0: RMSE_train=0.9147390720400589; RMSE_test=0.9186881194544265
Fold 1: RMSE_train=0.9160643857280082; RMSE_test=0.9133903868202118
Fold 2: RMSE_train=0.9159402241106509; RMSE_test=0.9138940864159748
Fold 3: RMSE_train=0.9158534751682688; RMSE_test=0.9142352887079381
Fold 4: RMSE_train=0.9150510848967625; RMSE_test=0.9174419496244308


Mean error on TRAIN: 0.9155296483887498
Mean error on  TEST: 0.9155299662045964
